In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from numpy import concatenate
from pandas import *
from sklearn.preprocessing import *
from sklearn.metrics import *
from tensorflow import keras
import tensorflow as tf
import time as t

2022-12-13 17:28:23.314645: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
dataset = read_csv("folder/subfolder/dataToModel_5min.csv",index_col= 0,sep=";")


In [4]:
dataset.head()

,rok,ms,dz,go,fco2_MQ,fco2_HQ,fco2_raw,glwody,T50cm,T2m,...,Ku,Ld,Lu,ppp,v,vdir,Tgrunt,vwc,PARd,PARu
1308,2013,1,5,13,-0.41,0.0,-0.41,4.7,0.6,0.3,...,1.8,309.6,316.7,1002.9,4.2,337.3,0.6,0.788,54.1,2.6
1309,2013,1,5,13,0.69,0.0,0.69,4.7,0.6,0.3,...,2.2,309.3,316.8,1002.9,4.8,337.8,0.6,0.788,65.9,3.1
1310,2013,1,5,13,0.51,0.0,0.51,4.7,0.6,0.3,...,1.6,310.0,316.7,1002.8,5.4,337.8,0.6,0.788,49.7,2.4
1311,2013,1,5,13,NaN,0.0,0.17,4.7,0.6,0.4,...,2.3,309.4,316.8,1002.7,5.1,333.1,0.7,0.788,67.2,3.3
1312,2013,1,5,13,1.23,0.0,1.23,4.7,0.6,0.3,...,2.0,309.1,316.7,1002.5,4.9,334.9,0.7,0.788,59.4,2.9


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 475336 entries, 1308 to 105407
Data columns (total 21 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   rok       475336 non-null  int64  
 1   ms        475336 non-null  int64  
 2   dz        475336 non-null  int64  
 3   go        475336 non-null  int64  
 4   fco2_MQ   298211 non-null  float64
 5   fco2_HQ   475336 non-null  float64
 6   fco2_raw  475336 non-null  float64
 7   glwody    475336 non-null  float64
 8   T50cm     475336 non-null  float64
 9   T2m       475336 non-null  float64
 10  Kd        475336 non-null  float64
 11  Ku        475336 non-null  float64
 12  Ld        475336 non-null  float64
 13  Lu        475336 non-null  float64
 14  ppp       475336 non-null  float64
 15  v         475336 non-null  float64
 16  vdir      475336 non-null  float64
 17  Tgrunt    475336 non-null  float64
 18  vwc       475336 non-null  float64
 19  PARd      475336 non-null  float64
 20  P

In [6]:
dataset.drop(columns  = ['dz','go','ms','dz','fco2_MQ','rok'], inplace = True)

In [7]:
dataset.head(2220)

,fco2_HQ,fco2_raw,glwody,T50cm,T2m,Kd,Ku,Ld,Lu,ppp,v,vdir,Tgrunt,vwc,PARd,PARu
1308,0.0,-0.41,4.7,0.6,0.3,25.6,1.8,309.6,316.7,1002.9,4.2,337.3,0.6,0.788,54.1,2.6
1309,0.0,0.69,4.7,0.6,0.3,31.7,2.2,309.3,316.8,1002.9,4.8,337.8,0.6,0.788,65.9,3.1
1310,0.0,0.51,4.7,0.6,0.3,23.5,1.6,310.0,316.7,1002.8,5.4,337.8,0.6,0.788,49.7,2.4
1311,0.0,0.17,4.7,0.6,0.4,32.6,2.3,309.4,316.8,1002.7,5.1,333.1,0.7,0.788,67.2,3.3
1312,0.0,1.23,4.7,0.6,0.3,28.5,2.0,309.1,316.7,1002.5,4.9,334.9,0.7,0.788,59.4,2.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22231,1.0,0.41,4.4,-6.3,-6.3,0.0,0.4,258.5,286.3,989.8,5.3,77.7,0.6,0.675,0.5,0.2
22232,1.0,0.39,4.4,-6.3,-6.3,0.0,0.4,259.3,286.4,990.0,5.8,80.4,0.6,0.675,0.8,0.3
22233,1.0,0.45,4.4,-6.3,-6.2,0.0,0.4,259.7,286.4,990.2,5.8,80.8,0.6,0.675,1.3,0.5
22234,1.0,0.58,4.4,-6.2,-6.2,0.0,0.4,260.2,286.5,990.4,5.9,80.3,0.6,0.675,1.6,0.6


In [8]:
values = dataset.values
values = values.astype('float64')

In [9]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
scaled = pd.DataFrame(scaled)

In [10]:
scaled.head(2220)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,0.491578,0.864865,0.400347,0.402135,0.021455,0.003073,0.508166,0.326026,0.542022,0.344262,0.936944,0.045045,0.800357,0.024668,0.001744
1,0.0,0.492158,0.864865,0.400347,0.402135,0.026567,0.003756,0.507224,0.326339,0.542022,0.393443,0.938333,0.045045,0.800357,0.030049,0.002080
2,0.0,0.492063,0.864865,0.400347,0.402135,0.019695,0.002732,0.509422,0.326026,0.540804,0.442623,0.938333,0.045045,0.800357,0.022662,0.001610
3,0.0,0.491884,0.864865,0.400347,0.403915,0.027321,0.003927,0.507538,0.326339,0.539586,0.418033,0.925278,0.049550,0.800357,0.030642,0.002214
4,0.0,0.492442,0.864865,0.400347,0.402135,0.023885,0.003415,0.506595,0.326026,0.537150,0.401639,0.930278,0.049550,0.800357,0.027085,0.001945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2215,1.0,0.492010,0.862069,0.280763,0.284698,0.000000,0.000683,0.347676,0.230817,0.382460,0.434426,0.215833,0.045045,0.598930,0.000228,0.000134
2216,1.0,0.492000,0.862069,0.280763,0.284698,0.000000,0.000683,0.350188,0.231131,0.384896,0.475410,0.223333,0.045045,0.598930,0.000365,0.000201
2217,1.0,0.492031,0.862069,0.280763,0.286477,0.000000,0.000683,0.351445,0.231131,0.387333,0.475410,0.224444,0.045045,0.598930,0.000593,0.000335
2218,1.0,0.492100,0.862069,0.282496,0.286477,0.000000,0.000683,0.353015,0.231444,0.389769,0.483607,0.223056,0.045045,0.598930,0.000730,0.000402


In [11]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 475336 entries, 1308 to 105407
Data columns (total 16 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   fco2_HQ   475336 non-null  float64
 1   fco2_raw  475336 non-null  float64
 2   glwody    475336 non-null  float64
 3   T50cm     475336 non-null  float64
 4   T2m       475336 non-null  float64
 5   Kd        475336 non-null  float64
 6   Ku        475336 non-null  float64
 7   Ld        475336 non-null  float64
 8   Lu        475336 non-null  float64
 9   ppp       475336 non-null  float64
 10  v         475336 non-null  float64
 11  vdir      475336 non-null  float64
 12  Tgrunt    475336 non-null  float64
 13  vwc       475336 non-null  float64
 14  PARd      475336 non-null  float64
 15  PARu      475336 non-null  float64
dtypes: float64(16)
memory usage: 61.7 MB


In [12]:
dataset.shape

(475336, 16)

In [13]:
   #for w in range(4,33,2):
   for w in range(1,5):
    start = t.time()
    list0 = []
    list1 = []
    i=0
    #w=6
    for window in scaled.rolling(window=w):
       # print(window.shape)
        if(window.shape==(w,window.shape[1])):
            i+=1
           # print("Number of window "+ str(i))
            list0.append(np.asarray(window.copy()))
            y=0
            shape = window.shape
            for index, test in window.iterrows() :
               # print(y)
                if (y==w-1) :
                   # print(test[0])
                    list1.append(test[0])
                    agroup=test[0]
                y+=1

    dataFromSensors=np.array(list0)
    np.save("dataFromSensors_"+str(w),dataFromSensors)
    labels=np.array(list1)
    np.save("labels_"+str(w),labels)
    stop=t.time()-start
    print(w,stop)

1 87.02590703964233
2 110.2208468914032
3 135.40596985816956
4 158.42479491233826


In [28]:
#print(list0)

In [29]:
#dataFromSensors=np.array(list0)

In [30]:
#np.save("dataFromSensors_"+str(w),dataFromSensors)

In [34]:
#w=32

In [35]:
#print(dataFromSensors)
#dataFromSensors=np.load("dataFromSensors_"+str(w)+".npy")

In [36]:
#np.info(dataFromSensors)

class:  ndarray
shape:  (475305, 32, 16)
strides:  (4096, 128, 8)
itemsize:  8
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x7fe3b4000000
byteorder:  little
byteswap:  False
type: float64


In [37]:
#print(list1)

In [38]:
#labels=np.array(list1)

In [40]:
#np.save("labels_"+str(w),labels)

In [41]:
labels=np.load("labels_"+str(w)+".npy")

In [42]:
#print(labels)

In [43]:
#np.info(labels)

class:  ndarray
shape:  (475305,)
strides:  (8,)
itemsize:  8
aligned:  True
contiguous:  True
fortran:  True
data pointer: 0x7fe344907000
byteorder:  little
byteswap:  False
type: float64
